In [1]:
# 从hrv_data_10s_fortrain.csv 文档中筛选以下几个人的数据（每个人数据条数大于1000，同时有着4种情绪标签）
# 张卜依, 朱必艳, 朱燕, 朱颖, 李佳宜, 梁怡, 王滋雨, 赵紫涵, 邓雯, 陈盈盈, 韩乐蓉
# 去掉每行的最后一个整数，然后保存为seleted10.csv的文件

In [4]:
import pandas as pd
import numpy as np

In [7]:
# 导入所有必要的库
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 加载数据
df_normalized = pd.read_csv('seleted11_normalized.csv')

# 准备特征列
feature_cols = [col for col in df_normalized.columns if col not in ['subject', 'emotion']]

# 创建并初始化 LabelEncoder
le = LabelEncoder()
le.fit(df_normalized['emotion'])

# 打印情绪标签编码映射
print("情绪标签编码映射:")
for i, emotion in enumerate(le.classes_):
    print(f"  '{emotion}': {i}")

# 创建 SVM 模型
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42, class_weight='balanced')

print(f"\n数据集大小: {df_normalized.shape}")
print(f"特征列: {feature_cols}")
print(f"被试数量: {df_normalized['subject'].nunique()}")
print("初始化完成！")

情绪标签编码映射:
  '平静': 0
  '悲伤': 1
  '愉悦': 2
  '焦虑': 3

数据集大小: (14221, 8)
特征列: ['RMSSD', 'pNN58', 'SDNN', 'SD1', 'SD2', 'SD1_SD2']
被试数量: 11
初始化完成！


In [9]:
# 对所有被试进行SVM分类评估

# 存储每个被试的准确率
subject_accuracies = {}
all_subjects = df_normalized['subject'].unique()

print("=" * 60)
print("开始对所有被试进行SVM分类评估")
print("=" * 60)

# 遍历每一个被试
for subject_name in all_subjects:
    # 1. 筛选当前被试的数据
    subject_data = df_normalized[df_normalized['subject'] == subject_name]
    
    # 2. 准备特征和标签
    X_subject = subject_data[feature_cols]
    y_subject_text = subject_data['emotion']
    
    # 使用之前创建的LabelEncoder将情绪标签转换为数字
    y_subject = le.transform(y_subject_text)
    
    # 3. 随机划分80/20训练集和测试集
    # stratify=y_subject确保在训练集和测试集中，各种情绪的比例与原始数据保持一致
    X_train_subj, X_test_subj, y_train_subj, y_test_subj = train_test_split(
        X_subject, 
        y_subject, 
        test_size=0.2, 
        random_state=42, 
        stratify=y_subject
    )
    
    # 4. 训练SVM模型（每个被试使用一个新的模型实例）
    subject_svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42, class_weight='balanced')
    subject_svm.fit(X_train_subj, y_train_subj)
    
    # 5. 在测试集上进行预测和评估
    y_pred_subj = subject_svm.predict(X_test_subj)
    accuracy_subj = accuracy_score(y_test_subj, y_pred_subj)
    
    # 存储结果
    subject_accuracies[subject_name] = accuracy_subj
    
    # 打印当前被试的结果
    print(f"\n被试: {subject_name}")
    print(f"  训练集大小: {len(X_train_subj)} 条")
    print(f"  测试集大小: {len(X_test_subj)} 条")
    print(f"  模型准确率: {accuracy_subj:.4f} ({accuracy_subj*100:.2f}%)")

# 6. 计算并打印所有被试的平均准确率
print("\n" + "=" * 60)
print("所有被试分类结果汇总")
print("=" * 60)

# 按准确率排序
sorted_subjects = sorted(subject_accuracies.items(), key=lambda x: x[1], reverse=True)

for rank, (subject_name, accuracy) in enumerate(sorted_subjects, 1):
    print(f"{rank:2d}. {subject_name:8s}: {accuracy:.4f} ({accuracy*100:.2f}%)")

mean_accuracy = np.mean(list(subject_accuracies.values()))
std_accuracy = np.std(list(subject_accuracies.values()))

print("\n" + "-" * 60)
print(f"平均准确率: {mean_accuracy:.4f} ({mean_accuracy*100:.2f}%)")
print(f"标准差: {std_accuracy:.4f}")
print(f"最高准确率: {max(subject_accuracies.values()):.4f}")
print(f"最低准确率: {min(subject_accuracies.values()):.4f}")
print("=" * 60)

开始对所有被试进行SVM分类评估

被试: 赵紫涵
  训练集大小: 864 条
  测试集大小: 216 条
  模型准确率: 0.4028 (40.28%)

被试: 朱燕
  训练集大小: 1156 条
  测试集大小: 289 条
  模型准确率: 0.3287 (32.87%)

被试: 陈盈盈
  训练集大小: 917 条
  测试集大小: 230 条
  模型准确率: 0.3000 (30.00%)

被试: 邓雯
  训练集大小: 951 条
  测试集大小: 238 条
  模型准确率: 0.3319 (33.19%)

被试: 朱必艳
  训练集大小: 818 条
  测试集大小: 205 条
  模型准确率: 0.3463 (34.63%)

被试: 张卜依
  训练集大小: 881 条
  测试集大小: 221 条
  模型准确率: 0.3167 (31.67%)

被试: 朱颖
  训练集大小: 839 条
  测试集大小: 210 条
  模型准确率: 0.3095 (30.95%)

被试: 王滋雨
  训练集大小: 1595 条
  测试集大小: 399 条
  模型准确率: 0.2556 (25.56%)

被试: 梁怡
  训练集大小: 1008 条
  测试集大小: 253 条
  模型准确率: 0.3755 (37.55%)

被试: 张卜依
  训练集大小: 881 条
  测试集大小: 221 条
  模型准确率: 0.3167 (31.67%)

被试: 朱颖
  训练集大小: 839 条
  测试集大小: 210 条
  模型准确率: 0.3095 (30.95%)

被试: 王滋雨
  训练集大小: 1595 条
  测试集大小: 399 条
  模型准确率: 0.2556 (25.56%)

被试: 梁怡
  训练集大小: 1008 条
  测试集大小: 253 条
  模型准确率: 0.3755 (37.55%)

被试: 韩乐蓉
  训练集大小: 1504 条
  测试集大小: 377 条
  模型准确率: 0.3952 (39.52%)

被试: 李佳宜
  训练集大小: 840 条
  测试集大小: 210 条
  模型准确率: 0.3857 (38.57%)

所有被试分类结果汇总
 1. 赵紫涵    